### What is Matrix Factorization?

- **Matrix factorization** is a mathematical technique commonly used in recommendation systems to factorize a user-item interaction matrix into lower-dimensional matrices representing users and items.
- The core concept behind matrix factorization is to approximate the original matrix by decomposing it into two or more matrices.
- This process helps uncover latent factors that influence user-item interactions.


#### Let's Breakdown the process of Matrix Factorization in Simple Steps

- User-Item Interaction Matrix:

 - In recommendation systems, you start with a user-item interaction matrix, often denoted as R.
 - Rows correspond to users, columns correspond to items, and the entries $R_{i,j}$ represent user ratings, interactions, or preferences for items.

 <center><img src="https://drive.google.com/uc?id=1gGxM-opp1666trtmfMlc6JY4Ue46imbn"></center>

- Decomposition:

 - Matrix factorization decomposes this user-item interaction matrix R into two lower-dimensional matrices, typically denoted as U (user matrix) and I (item matrix).

 <center><img src="https://drive.google.com/uc?id=1t3poDcWoe-ewAi_QCgEXe1SBoSgqZGOB"></center>

- Dimensions:

 - The dimensions of the user matrix U are M x K, where M is the number of users, and K is the number of latent factors. The item matrix I has dimensions of K x N, where N is the number of items.

- Objective Function:

 - The goal of matrix factorization is to find the matrices U and V such that the product U * V approximates the original matrix R.
 - To achieve this, an objective function is defined, often using a loss function like Mean Squared Error (MSE) or a variant of it.

 > $Loss = 𝚺 (R_{ij}-(U_{i}*I_{j}))^{2}$

 - Here, U_i and I_j are the latent factor vectors for the i-th user and j-th item, respectively.

- Optimization:

 - The matrices U and I are optimized to minimize the loss function. This is typically done using optimization techniques like Gradient Descent or Alternating Least Squares (ALS).

 - We'll use Gradient descent to optimize the loss function.

 - The optimization process updates the latent factor vectors in U and V iteratively to improve the approximation of the original matrix R.
 - The objective is to find U and V that provide the best fit to the observed user-item interactions.

- Prediction:

 - Once the optimization is complete, the factorized matrices U and V can be used to predict missing values in the original matrix R.
 - These predictions are used to generate recommendations for users.





In [160]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [161]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1' -O "movies.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1HOFWUAMFlYbd-gk1B2IyV2-hXDZI7gKR' -O "ratings.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1b7_yRRBs3s3atp1WQHN2GU577vxY8u_h' -O "users.csv"

--2025-12-10 10:45:51--  https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1
Resolving drive.google.com (drive.google.com)... 74.125.137.139, 74.125.137.138, 74.125.137.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download [following]
--2025-12-10 10:45:51--  https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 515699 (504K) [application/octet-stream]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 503.61K  --.-KB/s   

In [162]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')

In [163]:
movies.shape, ratings.shape, users.shape

((10329, 3), (105339, 4), (668, 3))

In [164]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [165]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [166]:
users.head()

,userId,age,time_spent_per_day
0,1,16,3.976315
1,2,24,1.891303
2,3,20,4.521478
3,4,23,2.095284
4,5,35,1.759860


In [167]:
ratings.head()
ratings = ratings[['userId', 'movieId', 'rating']]

In [168]:
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head()
rm = user_item_matrix

In [169]:
rm.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
rm_small = rm.copy()
rm_small = rm_small[rm_small.columns[:100]]
rm_small = rm_small.head(100)
rm_small.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,100,101,102,103,104,105,107,108,110,111
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [171]:
K = 2
P = np.random.normal(size = (rm_small.shape[0],K))
Q = np.random.normal(size= (rm_small.shape[1],K))

In [172]:
def matrix_factorization(R,P,Q,K, steps=10000, alpha=0.0002, beta=0.02):
    Q = Q.T
    eji = 0
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] == 0:
                    continue

                eji = R[i][j] - np.dot(P[i, :], Q[:, j])
                for k in range(K):
                    x = P[i][k]
                    P[i][k] += alpha * (2 * eji * Q[k][j] - beta * P[i][k])
                    Q[k][j] += alpha * (2 * eji * x - beta * Q[k][j])
    return P, Q.T



In [173]:
B, C = matrix_factorization(rm_small.values, P.copy(), Q.copy(), K)

In [174]:
print(np.dot(B[4], C[36]), rm_small.values[4, 36])
print(np.dot(B[1], C[0]), rm_small.values[1, 0])
print(np.dot(B[1], C[2]), rm_small.values[1, 2])
print(np.dot(B[3], C[17]), rm_small.values[3, 17])

5.079337016982378 0.0
4.913797385009593 5.0
3.4093496604988136 2.0
-1.4587418783517296 0.0


In [175]:
print(B.shape)
print(C.shape)

(100, 2)
(100, 2)


In [176]:
print(B)
print(C)

[[ 1.77433161  0.29873475]
 [ 1.333991   -1.74082777]
 [ 2.26109564 -0.70402922]
 [ 1.96603179  1.60671765]
 [-0.68791169 -3.34417302]
 [ 0.6731942   0.85404838]
 [ 2.24227935  0.00574247]
 [ 1.39708951 -1.60067178]
 [ 1.37971291 -0.92524861]
 [ 1.38932633 -1.88667061]
 [ 1.57982111 -0.39843969]
 [ 0.72964258 -2.02272264]
 [ 1.84018781  1.16186052]
 [ 0.45475128 -2.48453444]
 [ 2.17055578 -0.96593687]
 [ 2.1071823  -1.40909095]
 [ 2.51065378  1.34285773]
 [ 1.90629417  0.56267976]
 [ 0.25502325  1.2181132 ]
 [ 0.90599489 -0.98209135]
 [ 2.15443605  0.48206531]
 [ 1.46468264 -0.91506388]
 [ 1.69965498  0.37112741]
 [ 1.84605731 -1.85420887]
 [ 1.27119579 -1.07592681]
 [ 1.35579177  1.22735555]
 [-0.96214324 -2.09927804]
 [ 0.12245969 -2.2462444 ]
 [ 1.6970475  -0.22922307]
 [ 1.90067409 -0.4373256 ]
 [ 1.86284925 -0.969234  ]
 [ 1.86792511 -0.9687361 ]
 [ 2.18209362 -0.95684016]
 [ 2.04347122 -0.5131913 ]
 [ 2.47881351  0.65103545]
 [ 2.18852954  0.15036034]
 [-0.01130416 -1.96806705]
 

In [177]:
from sklearn.metrics import mean_squared_error
rm_  = np.dot(B, C.T)
mean_squared_error(rm_small.values, rm_)  


6.63829118122225

In [189]:
mean_squared_error(rm_small.values[rm_small.values > 0], rm_[rm_small.values > 0])

0.3227832496693763

In [178]:
rm_raw  = ratings.copy()

In [179]:
rm_raw.head()

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [180]:
!pip install cmfrec

In [181]:
from cmfrec import CMF
rm_raw.rename(columns={'userId': 'UserId', 'movieId': 'ItemId', 'rating': 'Rating'}, inplace=True)

In [182]:
rm_raw.head()

,UserId,ItemId,Rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [183]:
model = CMF(method = "als" ,k=2, lambda_=0.1,user_bias=False,item_bias=False,verbose=True)

In [184]:
model.fit(rm_raw)

Starting ALS optimization routine

Updating B ... done
Updating A ... done
	Completed ALS iteration  1

Updating B ... done
Updating A ... done
	Completed ALS iteration  2

Updating B ... done
Updating A ... done
	Completed ALS iteration  3

Updating B ... done
Updating A ... done
	Completed ALS iteration  4

Updating B ... done
Updating A ... done
	Completed ALS iteration  5

Updating B ... done
Updating A ... done
	Completed ALS iteration  6

Updating B ... done
Updating A ... done
	Completed ALS iteration  7

Updating B ... done
Updating A ... done
	Completed ALS iteration  8

Updating B ... done
Updating A ... done
	Completed ALS iteration  9

Updating B ... done
Updating A ... done
	Completed ALS iteration 10

ALS procedure terminated successfully


Collective matrix factorization model
(explicit-feedback variant)


In [185]:
model.A_.shape , model.B_.shape

((668, 2), (10325, 2))

In [186]:
model.A_

array([[ 0.6181083 , -1.296816  ],
       [ 1.0773844 ,  0.44746563],
       [ 0.7495232 , -0.2906971 ],
       ...,
       [ 0.72334623, -1.0197821 ],
       [ 0.64136606, -0.22247148],
       [-0.2533895 , -1.4885318 ]], dtype=float32)

In [187]:
rm__ = np.dot(model.A_, model.B_.T)
mean_squared_error(rm.values[rm > 0], rm__[rm > 0]) ** 0.5

3.8516142259951596

In [188]:
#Homework : Build a classifier/regressor using the users , movies , ratings data

# created 1 joined dataset
# rating will be the dependent variable
# column from the tables like age , time_spent , genres etc would be